In [8]:
import numpy as np
import gym
from gym import spaces
import pygame
import torch 

# Import packages
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from typing import List, Tuple
from matplotlib import animation

import collections
from collections import namedtuple, deque
import tqdm
import matplotlib.pyplot as plt
import random
import gymnasium as gym

from IPython.display import clear_output
from IPython import display
from gym.wrappers import FlattenObservation

torch.set_default_dtype(torch.float64)

In [15]:
class balloon3D:
    def __init__(self,width,length,height):
        # self.obs_size = obs_size
        self.x, self.y , self.z = 0,0,0
        self.height = height
        self.width = width
        self.length = length
        self.map = None

    def reset(self,start_x,start_y,start_z):
        self.x, self.y ,self.z = start_x,start_y,start_z

    def up(self):
        self.z = min(self.height-1,self.z+1)

    def down(self):
        self.z = max(self.z-1,0)

    def step(self):
        newx = int(max(0,min(self.x + self.map[0,self.x,self.y,self.z],self.width-1)))
        newy = int(max(0,min(self.y + self.map[1,self.x,self.y,self.z],self.length-1)))
        self.x = newx
        self.y = newy

    def generate_map(self,eta,mu):
        self.map = np.zeros((2,self.width,self.length,self.height))
        for i in range(self.width):
            for j in  range(self.length):
                for k in range(self.height):
                    for l in range(2):
                        r = np.random.random()
                        if(r>eta):
                            r= np.random.random()
                            if(r>mu):
                                self.map[l,i,j,k] = 1
                            else:
                                self.map[l,i,j,k] = -1
                            if j == self.length-1:
                                self.map[1,i,j,k] = -1
                            if j == 0:
                                self.map[1,i,j,k] = 1
                            if i == self.width-1:
                                self.map[0,i,j,k] = -1
                            if i == 0:
                                self.map[0,i,j,k] = 1


    def set_map(self,map):
        self.map = map

    def get_winds(self):
        # """r = self.obs_size
        # self.obs = np.zeros((2*r+1,2*r+1,self.height,2))
        # for i in range(self.x - r , self.x + r + 1):
        #     for j in range(self.y - r, self.y + r + 1):
        #         for k in range(0,self.height):
        #             for l in range(2):
        #                 self.obs[i - self.x + r, j- self.y + r,k,l]  = (self.map[i%self.width,j%self.length,k,l])
        # return self.obs"""
        self.obs= self.map
        return self.obs

    
    def render_obs(self):
        # r =self.obs_size*2+1
        for z in range(self.height):
            for j in range(self.length):
                s = ''
                for i in range(self.width):
                    l = ''
                    if(self.obs[0,i,j,z] == 0):
                        l += '.'
                    elif(self.obs[0,i,j,z] == 1):
                        l+='>'
                    elif(self.obs[0,i,j,z] == -1):
                        l+='<'

                    if(self.obs[1,i,j,z] == 0):
                        l += '.'
                    elif(self.obs[1,i,j,z] == 1):
                        l+='v'
                    elif(self.obs[1,i,j,z] == -1):
                        l+='^'                  
                    s+= l+ " "
                print(s)
            print(" ")
    

    
    def render(self):
        for z in range(self.height):
            for j in range(self.length):
                s = ''
                for i in range(self.width):
                    if self.x == i and self.y == j and self.z == z : 
                        s+= 'OO '
                    else:
                        l = ''
                        if(self.map[0,i,j,z] == 0):
                            l += '.'
                        elif(self.map[0,i,j,z] == 1):
                            l+='>'
                        elif(self.map[0,i,j,z] == -1):
                            l+='<'

                        if(self.map[1,i,j,z] == 0):
                            l += '.'
                        elif(self.map[1,i,j,z] == 1):
                            l+='v'
                        elif(self.map[1,i,j,z] == -1):
                            l+='^'
                        
                        s+= l+ " "
                print(s)
            print(" ")
                
        

In [16]:
bal = balloon3D(10,10,3)
bal.generate_map(0.4,0.5)
bal.render()

OO >v <v >v <v >v <v .v .v <v 
>. .v .. .^ >^ .. .. >. <. <. 
>. <^ <. <. <v .. >^ .v .v .. 
>v .. >. >^ <v <v .v >v .^ <. 
>. <. <. .. >. .^ >. .v .^ .. 
>^ .. <^ <. <. .^ >v >v >. <v 
>^ >^ .v >^ >. >^ .. .v <. <v 
>v .^ .v <. .. <. .. <v .. <^ 
>. .. <v >v .v .^ .v .. <^ <^ 
.. <^ >^ <^ >^ >^ .^ .. .. <^ 
 
>v >v <v .v >v <v .v <v .v <v 
>^ <. .v >. >v <v .v .. .^ <^ 
>^ .^ <v >^ >^ .^ .. .. <^ <v 
>^ .. <^ <. <. >. .. >. <^ <. 
>^ .^ >^ .. >v >v .^ <. >. <^ 
.. >. >^ <^ <^ <^ >^ <^ .. <^ 
>. .. .^ <. .v <. .^ >v >. <. 
>. >v >. .v >^ .. >v >^ .v <. 
>v <. .v >v >^ >v .. .. <. <^ 
>^ .. >^ <^ <^ >^ .. .. >^ <^ 
 
>v >v .. >v >v .v <v <v >v <v 
>. .^ .. <^ .^ .^ <^ .^ <. <. 
>^ >. <. >v .v <v >v >. >v <. 
>. .^ <. <. <v .v .. <. <^ <v 
.. >. .. .v .^ >v >^ <. >v <. 
>. >. .. .^ <^ >^ <^ <. >^ <^ 
>v <. .^ .. <^ <^ <. <^ >^ <v 
>^ <v .^ >^ <^ .. .. >^ >^ <^ 
>^ .^ .. >^ >. .^ .. .^ .^ <v 
>^ <^ >^ .. >^ .. <^ <^ .. <^ 
 


In [43]:
bal.step()
bal.render()

.. >v <v >v <v >v <v .v .v <v 
>. .v .. .^ >^ .. .. >. <. <. 
>. <^ <. <. <v .. >^ .v .v .. 
>v .. >. >^ <v <v .v >v .^ <. 
>. <. <. .. >. .^ >. .v .^ .. 
>^ .. <^ <. <. .^ >v >v >. <v 
>^ >^ .v >^ >. >^ .. .v <. <v 
>v .^ .v <. .. <. .. <v .. <^ 
>. .. <v >v .v .^ .v .. <^ <^ 
.. <^ >^ <^ >^ >^ .^ .. .. <^ 
 
>v >v <v .v >v <v .v <v .v <v 
>^ <. .v >. >v <v .v .. .^ <^ 
>^ .^ <v >^ >^ .^ .. .. <^ <v 
>^ OO <^ <. <. >. .. >. <^ <. 
>^ .^ >^ .. >v >v .^ <. >. <^ 
.. >. >^ <^ <^ <^ >^ <^ .. <^ 
>. .. .^ <. .v <. .^ >v >. <. 
>. >v >. .v >^ .. >v >^ .v <. 
>v <. .v >v >^ >v .. .. <. <^ 
>^ .. >^ <^ <^ >^ .. .. >^ <^ 
 
>v >v .. >v >v .v <v <v >v <v 
>. .^ .. <^ .^ .^ <^ .^ <. <. 
>^ >. <. >v .v <v >v >. >v <. 
>. .^ <. <. <v .v .. <. <^ <v 
.. >. .. .v .^ >v >^ <. >v <. 
>. >. .. .^ <^ >^ <^ <. >^ <^ 
>v <. .^ .. <^ <^ <. <^ >^ <v 
>^ <v .^ >^ <^ .. .. >^ >^ <^ 
>^ .^ .. >^ >. .^ .. .^ .^ <v 
>^ <^ >^ .. >^ .. <^ <^ .. <^ 
 


In [45]:
bal.get_winds()
bal.render_obs()

.. >v <v >v <v >v <v .v .v <v 
>. .v .. .^ >^ .. .. >. <. <. 
>. <^ <. <. <v .. >^ .v .v .. 
>v .. >. >^ <v <v .v >v .^ <. 
>. <. <. .. >. .^ >. .v .^ .. 
>^ .. <^ <. <. .^ >v >v >. <v 
>^ >^ .v >^ >. >^ .. .v <. <v 
>v .^ .v <. .. <. .. <v .. <^ 
>. .. <v >v .v .^ .v .. <^ <^ 
.. <^ >^ <^ >^ >^ .^ .. .. <^ 
 
>v >v <v .v >v <v .v <v .v <v 
>^ <. .v >. >v <v .v .. .^ <^ 
>^ .^ <v >^ >^ .^ .. .. <^ <v 
>^ .. <^ <. <. >. .. >. <^ <. 
>^ .^ >^ .. >v >v .^ <. >. <^ 
.. >. >^ <^ <^ <^ >^ <^ .. <^ 
>. .. .^ <. .v <. .^ >v >. <. 
>. >v >. .v >^ .. >v >^ .v <. 
>v <. .v >v >^ >v .. .. <. <^ 
>^ .. >^ <^ <^ >^ .. .. >^ <^ 
 
>v >v .. >v >v .v <v <v >v <v 
>. .^ .. <^ .^ .^ <^ .^ <. <. 
>^ >. <. >v .v <v >v >. >v <. 
>. .^ <. <. <v .v .. <. <^ <v 
.. >. .. .v .^ >v >^ <. >v <. 
>. >. .. .^ <^ >^ <^ <. >^ <^ 
>v <. .^ .. <^ <^ <. <^ >^ <v 
>^ <v .^ >^ <^ .. .. >^ >^ <^ 
>^ .^ .. >^ >. .^ .. .^ .^ <v 
>^ <^ >^ .. >^ .. <^ <^ .. <^ 
 


In [138]:
class BalEnv(gym.Env):
    def __init__(self, render_mode=None, s_x = 16 , s_y =16 , s_z = 4):
        self.balloon = balloon3D(s_x,s_y,s_z)

        self._target_location = [s_x-1,s_y-1,s_z-1]

        self.pos = np.zeros((1,s_x,s_y,s_z))

        self.observation_space = spaces.Box(-1, 1, shape=(3,s_x,s_y,s_z), dtype=int)

        self.action_space = spaces.Discrete(3)

    def reset(self):
        self.evo_time = 100

        self.reward = 0

        self.reward_x, self.reward_y = 0,0

        self.balloon.generate_map(0.35,0.5)

        self.time = 0

        self.balloon.reset(0,0,0)

        self._agent_location = [0,0,0]

        self.pos[0,self._agent_location[0],self._agent_location[1],self._agent_location[2]] = 10

        self.pos[0,self._target_location[0],self._target_location[1],self._target_location[2]] = -10

        observation = self._get_obs()

        return observation, {}

    def step(self, action):
        self.time += 1

        self.pos[0,self._agent_location[0],self._agent_location[1],self._agent_location[2]] = 0

        if action == 0:
            self.balloon.step()
        if action == 1:
            self.balloon.up()
        if action == 2:
            self.balloon.down()


        
        self._agent_location = np.array([self.balloon.x , self.balloon.y,self.balloon.z])

        self.pos[0,self._agent_location[0],self._agent_location[1],self._agent_location[2]] = 10

        terminated  = ((self._agent_location[0]==self._target_location[0])and(self._agent_location[1]==self._target_location[1]))
        
        distx = (self._target_location[0] - self._agent_location[0])
        disty = self._agent_location[1] - self._target_location[1]



        self.reward = -(np.sqrt((distx)**2 + (disty)**2))

        observation = self._get_obs()

        return observation, self.reward, terminated, self.time ==self.evo_time, {}
    
    def _get_obs(self):

        winds =  self.balloon.get_winds()
        obs = np.concatenate((self.pos, winds), axis=0)
        return  obs

    def render(self):
        self.balloon.render()
        
    def close(self):
        pass

In [139]:
env = BalEnv()
env.reset()
obs = torch.tensor([env._get_obs()])
print(obs.shape)

torch.Size([1, 3, 16, 16, 4])


In [140]:
a = env.action_space.sample()
print(a)
_,r,_,_,_ = env.step(a)
env.render()
print(r)

0
>v >v <v <v <v .v .. >v >v .v <v >v .v <v <v <v 
.. OO .v <. <. >^ <. >^ .^ <v .^ .^ <v <. <v <. 
>^ <^ <v .. >v .v >^ <v >^ >v >v >v <. <. >. <. 
>v >v <v <v <v <. <. >v .^ <. <^ >^ >^ <. >^ <^ 
>^ <^ <. <. .^ <^ >^ <. <v .. <v .v .^ >v <. <v 
>. >v .. <v .^ .^ <v >^ .v .. <v <^ <v >v <^ <^ 
>v <^ .^ >. >v .^ >^ >^ >. >^ >^ >v .. >^ <v <v 
>v <v >^ .. <v .. <. <^ <. .. .. >v <v >v <. <^ 
>v .v >. .^ <. .v <. .^ .^ <v <. >^ >^ <. >^ <v 
>. .^ >. .. <v .^ <v >. >. .v >. .v <. <^ <v .. 
>v >. <^ .v <v .v .. >. .v .. >^ <. >. .. .v <^ 
>^ .v .v <. <v .v <v <v >. >^ <. >. <v <. <. <v 
>. >^ <v <^ >^ >v .. .. >. .v <. <. .^ >v .^ <v 
>. <^ <v <v .^ .. .. >v .v <. >^ <^ .^ <^ .^ <. 
.. <. <v .^ <. <. <^ .v >^ <^ >v <. .. .. .v <v 
.. .^ .. <^ .^ .. <^ .^ .. <^ >^ >^ .^ <^ <^ <^ 
 
>v >v .v >v >v <v .v <v .v >v >v >v .v <v <v <v 
>v <v .^ .. >. .. >. >^ .^ <v <^ <. >^ .. .^ <^ 
>^ <v >v <^ >v >v .. .v >. >v >. <v .v .. >v <^ 
>^ <^ >v .v >. >. .^ .. .^ >^ .. >^ .^ >^ >v <. 
.. <^ <v >^ .v <

In [141]:
class Simple3DCNN(nn.Module):
    def __init__(self):
        super(Simple3DCNN, self).__init__()
        self.soft = torch.nn.Softmax(dim = -1)
        self.conv1 = nn.Conv3d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        self.adap_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        # Assuming input volume is 32x32x32, after two pooling layers it becomes 8x8x8
        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, 3)  # Assuming binary classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.adap_pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x =  F.relu(self.fc1(x))
        x =  F.relu(self.fc2(x))
        return self.soft(x)

class QNetwork:
    def __init__(self, env, lr, logdir=None):
        # Define Q-network with specified architecture
        self.net = Simple3DCNN()
        self.env = env
        self.lr = lr 
        self.logdir = logdir
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)

    def load_model(self, model_file):
        # Load pre-trained model from a file
        return self.net.load_state_dict(torch.load(model_file))

    def load_model_weights(self, weight_file):
        # Load pre-trained model weights from a file
        return self.net.load_state_dict(torch.load(weight_file))
    


In [142]:
state, _ = env.reset()
state = torch.tensor(state, dtype=torch.double).unsqueeze(0)
print(state.shape)

torch.Size([1, 3, 16, 16, 4])


In [143]:
class ReplayMemory:
    def __init__(self, env, memory_size=50000, burn_in=10000):
        # Initializes the replay memory, which stores transitions recorded from the agent taking actions in the environment.
        self.memory_size = memory_size
        self.burn_in = burn_in
        self.memory = collections.deque([], maxlen=memory_size)
        self.env = env

    def sample_batch(self, batch_size=32):
        # Returns a batch of randomly sampled transitions to be used for training the model.
        return random.sample(self.memory, batch_size)

    def append(self, transition):
        # Appends a transition to the replay memory.
        self.memory.append(transition)

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class DQN_Agent:

    def __init__(self, env, lr=5e-4, render=False):
        # Initialize the DQN Agent.
        self.env = env
        self.lr = lr
        self.policy_net = QNetwork(self.env, self.lr)
        self.target_net = QNetwork(self.env, self.lr)
        self.target_net.net.load_state_dict(self.policy_net.net.state_dict())  # Copy the weight of the policy network
        self.rm = ReplayMemory(self.env)
        self.burn_in_memory()
        self.batch_size = 32
        self.gamma = 0.99
        self.c = 0

    def burn_in_memory(self):
        # Initialize replay memory with a burn-in number of episodes/transitions.
        cnt = 0
        terminated = False
        truncated = False
        state, _ = self.env.reset()
        state = torch.tensor(state, dtype=torch.double).unsqueeze(0)

        # Iterate until we store "burn_in" buffer
        while cnt < self.rm.burn_in:
            # Reset environment if terminated or truncated
            if terminated or truncated:
                state, _ = self.env.reset()
                state = torch.tensor(state, dtype=torch.double).unsqueeze(0)
            
            # Randomly select an action (left or right) and take a step
            action = torch.tensor(self.env.action_space.sample()).reshape(1, 1)
            next_state, reward, terminated, truncated, _ = self.env.step(action.item())
            reward = torch.tensor([reward])
            if terminated:
                next_state = None
            else:
                next_state = torch.tensor(next_state, dtype=torch.double).unsqueeze(0)
                
            # Store new experience into memory
            transition = Transition(state, action, next_state, reward)
            self.rm.memory.append(transition)
            state = next_state
            cnt += 1

    def epsilon_greedy_policy(self, q_values, epsilon=0.05):
        # Implement an epsilon-greedy policy. 
        p = random.random()
        if p > epsilon:
            with torch.no_grad():
                return self.greedy_policy(q_values)
        else:
            return torch.tensor([[self.env.action_space.sample()]], dtype=torch.long)

    def greedy_policy(self, q_values):
        # Implement a greedy policy for test time.
        return torch.argmax(q_values)
        
    def train(self):
        # Train the Q-network using Deep Q-learning.
        state, _ = self.env.reset()
        state = torch.tensor(state, dtype=torch.double).unsqueeze(0)
        terminated = False
        truncated = False

        # Loop until reaching the termination state
        while not (terminated or truncated):
            with torch.no_grad():
                q_values = self.policy_net.net(state)

            # Decide the next action with epsilon greedy strategy
            action = self.epsilon_greedy_policy(q_values).reshape(1, 1)
            
            # Take action and observe reward and next state
            next_state, reward, terminated, truncated, _ = self.env.step(action.item())
            reward = torch.tensor([reward])
            if terminated:
                next_state = None
            else:
                next_state = torch.tensor(next_state, dtype=torch.double).unsqueeze(0)

            # Store the new experience
            transition = Transition(state, action, next_state, reward)
            self.rm.memory.append(transition)

            # Move to the next state
            state = next_state

            # Sample minibatch with size N from memory
            transitions = self.rm.sample_batch(self.batch_size)
            batch = Transition(*zip(*transitions))
            non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool)
            non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
            state_batch = torch.cat(batch.state)
            action_batch = torch.cat(batch.action)
            reward_batch = torch.cat(batch.reward)

            # Get current and next state values
            state_action_values = self.policy_net.net(state_batch).gather(1, action_batch) # extract values corresponding to the actions Q(S_t, A_t)
            next_state_values = torch.zeros(self.batch_size)
            
            with torch.no_grad():
                # no next_state_value update if an episode is terminated (next_satate = None)
                # only update the non-termination state values (Ref: https://gymnasium.farama.org/tutorials/gymnasium_basics/handling_time_limits/)
                next_state_values[non_final_mask] = self.target_net.net(non_final_next_states).max(1)[0] # extract max value
                
            # Update the model
            expected_state_action_values = (next_state_values * self.gamma) + reward_batch
            criterion = torch.nn.MSELoss()
            loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
            self.policy_net.optimizer.zero_grad()
            loss.backward()
            self.policy_net.optimizer.step()

            # Update the target Q-network in each 50 steps
            self.c += 1
            print(self.c)
            if self.c % 50 == 0:
                # print(self.test())
                self.target_net.net.load_state_dict(self.policy_net.net.state_dict())

    def test(self, n = 30,model_file=None):
        # Evaluates the performance of the agent over 20 episodes.
        rewards = []
        for i in range(n):
            max_t = 1000
            state, _ = self.env.reset()
            print(i)
            for t in range(max_t):
                state = torch.from_numpy(state).double().unsqueeze(0)
                with torch.no_grad():
                    q_values = self.policy_net.net(state)
                action = self.greedy_policy(q_values)
                state, reward, terminated, truncated, _ = self.env.step(action.item())
                if terminated or truncated:
                    rewards.append(reward)
                    break

        return np.average(rewards)

In [147]:
training.test()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


-19.346715923037028

In [144]:
training = DQN_Agent(env=env)

In [145]:
training.train()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
